<a href="https://colab.research.google.com/github/emmanuellaee/NLP_Final_Project/blob/main/NLP_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Question Answering: In this task, you’ll work with provided sentences
#or text samples from which questions are asked and must be answered.
#Here are some good question-answering datasets:
#Stanford Question Answering Dataset (SQuAD): Focuses on
#answering questions related to Wikipedia articles.
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving train-v2.0.json to train-v2.0.json


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dev-v2.0.json to dev-v2.0.json


In [ ]:
train_df = pd.read_json("train-v2.0.json")
dev_df = pd.read_json("dev-v2.0.json")

# Display the first few rows of the dataset
print("Training set:")
print(train_df.head())
print("\nDev set:")
print(dev_df.head())

Training set:
  version                                               data
0    v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1    v2.0  {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2    v2.0  {'title': 'Sino-Tibetan_relations_during_the_M...
3    v2.0  {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4    v2.0  {'title': 'The_Legend_of_Zelda:_Twilight_Princ...

Dev set:
  version                                               data
0    v2.0  {'title': 'Normans', 'paragraphs': [{'qas': [{...
1    v2.0  {'title': 'Computational_complexity_theory', '...
2    v2.0  {'title': 'Southern_California', 'paragraphs':...
3    v2.0  {'title': 'Sky_(United_Kingdom)', 'paragraphs'...
4    v2.0  {'title': 'Victoria_(Australia)', 'paragraphs'...


In [ ]:
print(train_df.info())
print(train_df.head())
print(train_df["data"][0]["paragraphs"][0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  442 non-null    object
 1   data     442 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB
None
  version                                               data
0    v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1    v2.0  {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2    v2.0  {'title': 'Sino-Tibetan_relations_during_the_M...
3    v2.0  {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4    v2.0  {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
{'qas': [{'question': 'When did Beyonce start becoming popular?', 'id': '56be85543aeaaa14008c9063', 'answers': [{'text': 'in the late 1990s', 'answer_start': 269}], 'is_impossible': False}, {'question': 'What areas did Beyonce compete in when she was growing up?', 'id': '56be85543aeaaa14008c9065', 'answers': [{'text': 'singing and danc

In [ ]:
!pip install transformers

In [ ]:
import os


def clean_text(text):
    # have the text be in lowercase
    cleaned_text = text.lower()
    return cleaned_text

# format data to separate the different features into separate columns
def extract_data(df):
    data = []
    for article in df["data"]:
        title = article["title"]
        for paragraph in article["paragraphs"]:
            context = clean_text(paragraph["context"])
            for qanda in paragraph["qas"]:
                question = clean_text(qanda["question"])
                for answer in qanda["answers"]:
                    sentence = clean_text(answer["text"])
                    data.append({
                        "id": qanda["id"],
                        "title": title,
                        "context": context,
                        "question": question,
                        "answer": sentence,
                        "answer_start": answer["answer_start"],
                        "is_impossible": qanda["is_impossible"]
                    })
    return data

In [ ]:
#extract the data for train and dev datasets
train_prep = extract_data(train_df)
dev_prep = extract_data(dev_df)

#create DataFrames of the extracted data
train_prep_df = pd.DataFrame(train_prep)
dev_prep_df = pd.DataFrame(dev_prep)

#show head
print("Train DataFrame:")
print(train_prep_df.head())

print("\nDev DataFrame:")
print(dev_prep_df.head())

Train DataFrame:
                         id    title  \
0  56be85543aeaaa14008c9063  Beyoncé   
1  56be85543aeaaa14008c9065  Beyoncé   
2  56be85543aeaaa14008c9066  Beyoncé   
3  56bf6b0f3aeaaa14008c9601  Beyoncé   
4  56bf6b0f3aeaaa14008c9602  Beyoncé   

                                             context  \
0  beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...   
1  beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...   
2  beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...   
3  beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...   
4  beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...   

                                            question               answer  \
0           when did beyonce start becoming popular?    in the late 1990s   
1  what areas did beyonce compete in when she was...  singing and dancing   
2  when did beyonce leave destiny's child and bec...                 2003   
3      in what city and state did beyonce  grow up?        houston, texas   
4         in which d

In [ ]:
from transformers import RobertaTokenizerFast

#load a pre-trained RoBERTa tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def preprocess(example):
  question = [q.lstrip() for q in example["question"]]
  # Tokenize context and question
  tokenized_input = tokenizer(
      question,
      example["context"] ,
      truncation="only_second",  # Truncate the context if it exceeds max length
      # the max length can be reduced to 384 for computational efficiency
      max_length=384,             # RoBERTa supports up to 512 tokens
      padding="max_length",      # Pad to max length
      return_offsets_mapping=True
      )

  offset_mapping = tokenized_input.pop("offset_mapping")
  answers = example["answers"]
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = tokenized_input.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

  tokenized_input["start_positions"] = start_positions
  tokenized_input["end_positions"] = end_positions
  return tokenized_input

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset

# extract answer_start and text
def convert_ans(row):
    start = row[0]
    text = row[1]
    # return dictionary containing keys and extracted data from each row
    return {
        'answer_start': [start],
        'text': [text]
    }

# shuffle and apply 'convert_ans'
train = train_prep_df.sample(frac=1, random_state=42)
train['answers'] = train[['answer_start', 'answer']].apply(convert_ans, axis=1)
dev = dev_prep_df.sample(frac=1, random_state=42)
dev['answers'] = dev[['answer_start', 'answer']].apply(convert_ans, axis=1)

# convert dataframe to dataset to utilize map function
train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)

In [ ]:
train_tokenized_df = train_dataset.map(preprocess, batched=True, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/86821 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="qa_roberta_model",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=,
    per_device_eval_batch_size=,
    num_train_epochs=3,
    weight_decay=0.01,
)